In [ ]:
from imports import *
from centerlining_utils import *
from config_settings import *

In [ ]:
# Hide all markdown cells (including the space they occupy) using CSS
# This only happens when the notebook is ran in app mode
if ('/addon/' or '/apps/') in jupyter_notebook_url:
    display(HTML('''<style>
    .jp-Cell.jp-Notebook-cell.jp-MarkdownCell { display: none !important; }
    </style>'''))

# Logging

In [ ]:
log_handler = RotatingFileHandler(log_file, mode='a', maxBytes=5*1024*1024, 
                                 backupCount=2, encoding=None, delay=0)


# Create a Formatter
formatter = logging.Formatter('%(asctime)s %(levelname)-8s %(message)s')

# Set the Formatter for the handler
log_handler.setFormatter(formatter)

# Create a logger
app_log = logging.getLogger('root')
app_log.setLevel(logging.INFO)

# Add the handler to the logger
app_log.addHandler(log_handler)

# Initialized Variables

In [ ]:
speed_tag_df = pd.DataFrame()
unique_grades_list = []
input_widgets = {}
centerlining_table = pd.DataFrame(columns=column_order)
current = pd.DataFrame()
previous = pd.DataFrame()
selected_grade = str()
last_updated = str()
current_data = pd.DataFrame()
grades = list()
tags = list()
running_metrics = pd.DataFrame()
tag_search_df = pd.DataFrame()
centerline_metadata = pd.DataFrame()
csv_file_upload = pd.DataFrame()
tree = ''
data = pd.DataFrame()
required_columns = ["Tag Name", "Friendly Name", "Category"]
optional_columns = ["Path", "Asset", "Datasource Name", "Datasource Class", "Datasource ID"]


## Folder Directory Information

In [ ]:
# Get the current working directory
current_directory = os.getcwd()
# Create the full path for the new folder
CENTERLINE_FOLDER_PATH = os.path.join(current_directory, CENTERLINE_FOLDER)
os.makedirs(CENTERLINE_FOLDER_PATH, exist_ok=True)
centerlining_tree_selections = get_centerlining_sets(CENTERLINE_FOLDER_PATH)

# App (UI) Layout / Components

## Main Page

In [ ]:
#App Header
header = v.ToolbarTitle( children=['Seeq Centerline App'])

first_text = v.Html(tag='tr', children=['Please select a grade to view Centerline Results.'])
table_title = v.Html(tag='h2', children=['Selected grade will display here.'])
table_row = v.Flex(xs12=True, class_='flex-grow-1', style_='min-height: 500px;', children=[first_text])

select_grade_button = v.Btn(class_="white--text",color=app_color, children=['Select Grade'])
view_tag_in_wb_button = v.Btn(class_="white--text",color=app_color, children=['View Tag/Limits'])
create_or_manage_tree_button = v.Btn(class_="white--text",color=app_color, children=['Create or Manage Centerline Tree'])

# Top table that shows previous month run information
current_grade_metrics_datatable = v.DataTable(
        headers=[{"text": col, "value": col} for col in current_data.columns],
        items=current_data.to_dict('records')
    )

### Primary Centerline Table

In [ ]:
grid_options = {'defaultColDef': {'sortable': 'true', 'filter': 'true', 'resizable': 'true', 'editable': True},
               'columnDefs' : [{'field': c, 'cellStyle': cell_style if c == 'Priority' else None} for c in centerlining_table.columns]}

css_rules = """
.priority-high {
    background-color: red;
}
.priority-medium {
    background-color: orange;
}
"""

ccf = """function(params) { 
    if (params.value >= 2) {
        return 'priority-high'; 
    } else if (params.value >= 1) {
        return 'priority-medium';
    }
    return '';
}"""

grid_options['columnDefs'] = [{'field': c, 'cellClass': ccf if c == 'Priority' else ''} for c in centerlining_table.columns]

g = Grid(width='100%',
         height = 5000,
         grid_data=centerlining_table,
         grid_options=grid_options,
         theme='ag-theme-balham',
         quick_filter=True,
         css_rules=css_rules,  # add css_rules
         sync_grid=True,
         export_csv=True,
         export_excel=True,
         show_toggle_delete=False,
         columns_fit= 'size_to_fit',
         index=False,
         keep_multiindex=False
        )

## Misc UI Components

### Pop Up Messages

In [ ]:
error_card = v.Card(children=[])
error_dialog = v.Dialog(
    max_width='300',
    v_model = None,
    children=[error_card],
)

card = v.Card(children=[])
dialog = v.Dialog(
    max_width='300',
    v_model = None,
    children=[card],
)

### Outputs

In [ ]:
# Output for URL Hyperlink, CSV Uplodad, and delete tree
output = Output()
csv_output = Output()
delete_tree_output = Output()

# Outputs for tree creation
output_grades = Output()
output_downtimes = Output()
output_speed_tag = Output()

## Dialog for "Select Grade"

In [ ]:
tree_selection_input = v.Combobox(label='Select Centerlining Set', items=centerlining_tree_selections, v_model=None)
grade_selection_input = v.Combobox(label='Select Grade', items=grades, v_model=None)

execute_grade_selection_button = v.Btn(class_="white--text",color=app_color, children=['Execute'])

#Loading Message
please_wait = v.Html(tag='p', children=['Loading... This may take a moment to collect the relevant data.'], style_='display: none')

dialog_grade_selection = v.Dialog(v_model=False, children=[
    v.Card(children=[
        v.CardTitle(children=['Select Centerline Tree & Grade']),
        v.CardText(children=[tree_selection_input,
            grade_selection_input,
            please_wait
        ]),
        v.CardActions(children=[execute_grade_selection_button])
    ])
])

## Dialog for "Create or Manage Cenerline Tree"

In [ ]:
upload_or_manage_selector = v.Combobox(label='Upload or Manage', items=['Upload', 'Manage'], v_model='Upload')
upload_or_manage_selector_col = v.Col(children=[upload_or_manage_selector])

### Create Tree

In [ ]:
asset_tree_name_input= v.TextField(label='Enter the name for this Centerlining Data Set; e.g., Plant A Paper Machine 1', v_model='', disabled=False, class_='ma-10')

help_csv_format_button = v.Btn(class_="white--text",color=app_color, children=['CSV Formatting Guidelines'])
execute_new_tree_button = v.Btn(class_="white--text",color=GREEN, children=['Create Centerline Tree'], style_='width: auto', disabled = True)
find_all_grades_button = v.Btn(class_="white--text",color=app_color, children=['Find All Grades']) 
find_all_modes_button = v.Btn(class_="white--text",color=app_color, children=['Find All Tag Values'])        
find_speed_tag_button = v.Btn(class_="white--text",color=app_color, children=['Find Speed Tag']) 

help_dialog = v.Dialog(#width='300', 
                       v_model=False, children=[
    v.Card(children=[
        v.CardTitle(children=['Help Information']),
        v.CardText(children=['Required Columns: "Tag Name", "Friendly Name", "Category"\nOptional Columns: "Path", "Asset", "Datasource Name", "Datasource Class", "Datasource ID"']),
    ])
])

# File Upload Widget
file_upload = widgets.FileUpload(
    accept='.csv',  # Accepted file extension
    multiple=False,  # True to accept multiple files upload else False
    description='Upload CSV',  # Add title to the button
    layout= widgets.Layout(width='auto')
)

# Creation Widgets
string_or_number_selector = v.Combobox(label='Value Represents', items=['String', 'Number'], v_model='String')
input_value_uptime = v.TextField(
            label=f'Enter the exact value that represents UPTIME.',
            outlined=True,
            v_model=None,
            disabled = True
        )

grade_tag_item_selector = spy.widgets.SeeqItemSelect(
                    title = '<H5>Click Find All Grades after you have selected your grade signal.</H5>',
                    show_fields=['Name', 'Datasource Dropdown'],
                    multi_select=False,
                    type_options=['Signal'],
                    item_type='Signal',
                    )

downtime_tag_item_selector = spy.widgets.SeeqItemSelect(
                    title = '<H5>Click Find All Tag Values after you have selected your Uptime/Downtime signal.</H5>',
                    show_fields=['Name', 'Datasource Dropdown'],
                    multi_select=False,
                    type_options=['Signal'],
                    item_type='Signal',
                    )

speed_tag_item_selector = spy.widgets.SeeqItemSelect(
                    title = '<H5>Click Find Speed Tag after you have selected the tag that measures process/machine speed.</H5>',
                    show_fields=['Name', 'Datasource Dropdown'],
                    multi_select=False,
                    type_options=['Signal'],
                    item_type='Signal',
                    )
grade_tag_item_selector.layout.width='500px'
downtime_tag_item_selector.layout.width='500px'
speed_tag_item_selector.layout.width='500px'



def create_attribute_panel(name, description, default, input_type='text'):
    # Choose the correct vuetify component based on the input_type
    if input_type == 'integer':
        input_widget = v.TextField(
            label=f'Enter a number (e.g., {default})',
            type='number',
            outlined=True,
            v_model=None  # Initialize the v_model to None
        )
    elif input_type == 'string':
        input_widget = v.TextField(
            label=f'Enter text (e.g., {default})',
            outlined=True,
            v_model=None  # Initialize the v_model to None
        )
    elif input_type == 'grade_tag_selector':
        input_widget= v.Container(fluid=True, class_='pt-10', children=[
            v.Row(children=[grade_tag_item_selector]),
            v.Row(children=[output_grades]),
            v.Row(children=[find_all_grades_button])
        ])
    elif input_type == 'downtime_tag_selector':
        input_widget= v.Container(fluid=True, class_='pt-10', children=[
            v.Row(children=[downtime_tag_item_selector]),
            v.Row(children=[output_downtimes]),
            v.Col(cols = "6", children=[find_all_modes_button]),
            v.Col(cols = "6", children = [input_value_uptime]), 
            v.Col(cols = "6", children = [string_or_number_selector])
        ])
    elif input_type == 'speed_tag_selector':
        input_widget= v.Container(fluid=True, class_='pt-10', children=[
            v.Row(children=[speed_tag_item_selector]),
            v.Row(children=[output_speed_tag]),
            v.Row(children=[find_speed_tag_button])
        ])
    else:
        raise ValueError("input_type must be 'integer' or 'string' or 'grade_selector', 'downtime_selector'")
    
    panel = v.ExpansionPanel(children=[
        v.ExpansionPanelHeader(children=[name]),
        v.ExpansionPanelContent(children=[
            v.CardText(children=[description]),
            input_widget
        ])
    ])
    
    return panel, input_widget



# List of attributes with their attribute_types
# ('Simple Name', 'Description', 'example default value', 'attribute_type')
attributes = [
    ('Specify Grade Tag', 'Search for and select the Grade tag; must be a step signal.', 0, 'grade_tag_selector'),
    ('Specify Uptime/Downtime Tag', 'Search for and select the uptime/downtime tag; must be a step signal.', 0, 'downtime_tag_selector'),
    ('Specify Speed Tag', 'Search for and select the Speed tag.', 0, 'speed_tag_selector'),
    ('Upper Speed Limit Filter', 'Only data within the Upper and Lower Speed Limit Filter limits will be considered for limit creation.', 3500, 'integer'),
    ('Lower Speed Limit Filter', 'Only data within the Upper and Lower Speed Limit Filter limits will be considered for limit creation.', 2800, 'integer'),
    ('% Uptime Filter', 'Only grade runs with % Uptime greater than this threshold will be considered for limit creation', 90,'integer'),
    ('StdDev Multiplier (Inner)', 'StdDev Inner Multiplier', 1, 'integer'),
    ('StdDev Multiplier (Outer)', 'StdDev Outer Multiplier', 3, 'integer'),
    ('Remove Shorter Than (Shortest Grade Run)', 'Shortest expected grade run', '30 min', 'string'),
    ('Remove Longer Than (Longest Grade Run)', 'Longest expected grade run', '100 d', 'string'),
    ('Number of Previous Grade Runs for Limit Creation', 'Number of previous grade runs to consider', 3, 'integer'),
    ('Lookback Range for Previous Grade Runs', 'How far to look back for previous grade runs', '100 d', 'string'),
    ('Sampling Rate', 'Sampling rate to calculate limits', '30 min', 'string')
]

# Create the UI elements for each attribute and store the input widgets
attribute_panels = []
for attr in attributes:
    panel, input_widget = create_attribute_panel(*attr)
    attribute_panels.append(panel)
    input_widgets[attr[0]] = input_widget  # Store the widget with the name as the key


# Combine all attribute panels into one expansion panels group
attributes_expansion_panels = v.ExpansionPanels(children=attribute_panels, accordion=True)

# Create the higher-level panel to contain all the attribute panels
configuration_panel = v.ExpansionPanel(children=[
    v.ExpansionPanelHeader(children=['Centerlining Configuration']),
    v.ExpansionPanelContent(children=[attributes_expansion_panels])
])

# Combine the high-level panel into an Expansion Panels group
configuration_panels_group = v.ExpansionPanels(children=[
    configuration_panel
], accordion=True)


dialog_tree_upload_view_Title = v.CardTitle(children=['Upload a CSV with Centerlining Tags'], class_='ma-10')
dialog_tree_upload_view_Text = v.CardText(children=[v.Col(children = [v.Row(children = [asset_tree_name_input]),
                                                                     v.Row(children = [ csv_output])])])
dialog_tree_upload_view_Actions = v.CardActions(children=[
                                                    v.Row(children = [
                                                        v.Col(cols="2", children=[file_upload]),  # Assign half the width of the row to file_upload
                                                        v.Col(cols = "6", children = [configuration_panels_group]),
                                                        v.Col(cols="2", children=[execute_new_tree_button]),  # Assign the other half to execute_new_tree_button
                                                    ], class_='ma-10')
                                                ])

dialog_tree_upload = v.Dialog(v_model=False, children=[
    v.Card(children=[
        help_dialog,
        v.Row(justify='center',children = [upload_or_manage_selector_col,
        v.Col(children = [help_csv_format_button])], class_='ma-10'),
        dialog_tree_upload_view_Title,
        dialog_tree_upload_view_Text,
        dialog_tree_upload_view_Actions
    ])
])



dialog_tree_upload_or_manage = v.Dialog(v_model=False, children=[ #dialog_tree_upload
    v.Card(children=[
        help_dialog,
        v.Row(justify='center',children = [upload_or_manage_selector_col,
        v.Col(children = [help_csv_format_button])], class_='ma-10'),
        dialog_tree_upload_view_Title,
        dialog_tree_upload_view_Text,
        dialog_tree_upload_view_Actions
    ])
])

### Manage Tree

In [ ]:
delete_tree_selection_input = v.Combobox(label='Select Centerlining Set to Delete', items=centerlining_tree_selections, v_model=None)

execute_delete_tree_button = v.Btn(class_="white--text",color=RED, children=['Delete Centerline Tree'], style_='width: auto', disabled = True)

dialog_tree_manage_view_Title = v.CardTitle(children=['Delete Existing Centerline Trees'], class_='ma-10')
dialog_tree_manage_view_Text = v.CardText(children=[delete_tree_selection_input, delete_tree_output], class_='ma-10')
dialog_tree_manage_view_Actions = v.CardActions(children=[v.Row(children = [v.Col(cols = '6', children = [execute_delete_tree_button])], class_='ma-10')])
dialog_tree_manage = v.Dialog(v_model=False, children=[
    v.Card(children=[
        upload_or_manage_selector_col,
        dialog_tree_manage_view_Title,
        dialog_tree_manage_view_Text,
        dialog_tree_manage_view_Actions
    ])
])

## Dialog for "View Tag/Limits"

In [ ]:
tag_selection_input = v.Combobox(label='Select Tag', items=tags, v_model=None)

execute_view_tag_button = v.Btn(class_="white--text",color=app_color, children=['Execute'])

dialog_view_tag = v.Dialog(v_model=False, children=[
    v.Card(children=[
        v.CardTitle(children=['Select a tag to generate link. Limits will be viewed against the selected grade.']),
        v.CardText(children=[tag_selection_input, output]),
        v.CardActions(children=[execute_view_tag_button])
    ])
])

## App Layout

In [ ]:
top_table_container = v.Container(fluid=True, class_='pt-10', children=[
            v.Row(children=[current_grade_metrics_datatable]),
        ])


main_container = v.Container(fluid=True, class_='pt-10', children=[
            v.Row(children=[
                v.Col(children=[select_grade_button]),
                v.Col(children=[view_tag_in_wb_button]),
                v.Col(children=[create_or_manage_tree_button]),
                top_table_container
            ]),
            v.Row(children=[table_title]),
            table_row,
            dialog_view_tag,
            dialog_grade_selection,
            dialog_tree_upload_or_manage
        ])


app = v.Layout(
    _metadata={'mount_id': 'content-main'},
    children=[
        v.AppBar(app=True, dark=True, color=app_color, children=[
            header
        ]),
        main_container,
        error_dialog,
        dialog
    ]
)

# Functions

## Utility / Misc

In [ ]:
# Pop up with an error. Best used with a try: except:
def show_error_message(message):
    error_card.children = [
            v.CardTitle(class_='headline gray lighten-2', primary_title=True, children=['Error']),
            v.CardText(children=[message]), 
    ]
    
    error_dialog.v_model = True  # Show the dialog
    
def show_message(title, message):
    card.children = [
            v.CardTitle(class_='headline gray lighten-2', primary_title=True, children=[title]),
            v.CardText(children=[message]), 
    ]
    
    dialog.v_model = True  # Show the dialog
    
    
def validate_columns(df):
    """ Validate the columns of the DataFrame and return a tuple (is_valid, message) """
    missing_required = [col for col in required_columns if col not in df.columns]
    invalid_columns = [col for col in df.columns if col not in required_columns + optional_columns]

    if missing_required or invalid_columns:
        message_parts = []
        if missing_required:
            message_parts.append(f"Missing required columns: {', '.join(missing_required)}")
        if invalid_columns:
            message_parts.append(f"Invalid columns: {', '.join(invalid_columns)}")
        message_parts.append(f"Allowed columns are: {', '.join(required_columns + optional_columns)}")
        return False, "; ".join(message_parts)

    return True, "Columns are valid."

def check_all_inputs_specified(input_widgets):
    global speed_tag_df
    for widget in input_widgets.values():
        if isinstance(widget, v.TextField):
            # Check v.TextFields for non-empty input
            if widget.v_model is None or widget.v_model == '':
                return False
        elif isinstance(widget, v.Container):
            if not grade_tag_item_selector.selected_value:
                #show_error_message(f'You must search for and select a single Grade tag.')
                return False
            if not downtime_tag_item_selector.selected_value:
                #show_error_message(f'You must search for and select a single Uptime / Downtime tag.')
                return False
            if input_value_uptime is None or input_value_uptime.v_model == '':
                return False
            if len(speed_tag_df) == 0:
                return False
        else:
            # Handle other widget types or raise an error
            raise ValueError(f"Unhandled widget type: {type(widget)}")
    return True

def get_all_input_values(input_widgets):
    values = {}
    for name, widget in input_widgets.items():
        if isinstance(widget, v.TextField):  # Check if the widget is a v.TextField
            values[name] = widget.v_model  # Store the user-input value
    return values

def create_centerlining_tree():
    global unique_grades_list
    global csv_file_upload
    global speed_tag_df
    global input_widgets
    global created_trees
    with csv_output:
        name_of_tree = asset_tree_name_input.v_model
        
        display(f"Beginning to build centerlining Set: {name_of_tree}... Please keep this app open until complete.")
        
        tags_df = csv_file_upload
        grade_tag_df = pd.DataFrame([grade_tag_item_selector.selected_value])
        downtime_tag_df = pd.DataFrame([downtime_tag_item_selector.selected_value])
        uptime_value = input_value_uptime.v_model
        input_config_values = get_all_input_values(input_widgets)
        string_or_number = string_or_number_selector.v_model
        try:
            create_tree(name_of_tree, tags_df, speed_tag_df, input_config_values, unique_grades_list, grade_tag_df, downtime_tag_df, string_or_number, uptime_value, centerlining_workbook, centerlining_datasource, CENTERLINE_FOLDER_PATH)
            show_message('Congrats!', 'Your Centerlining Dataset has been successfully processed. You may now select it to view the limits.')

            ## UPDATE THE DROPDOWN SO THAT USER CAN SEE NEW TREE IMMEDIATELY.
            # Get the latest sets
            centerlining_tree_selections = get_centerlining_sets(CENTERLINE_FOLDER_PATH)

            # Directly update the tree_selection_input element's items
            tree_selection_input.items = centerlining_tree_selections
            delete_tree_selection_input.items = centerlining_tree_selections


        except Exception as e:
            show_error_message(f"Error occurred while creating Centerline Tree: {e}")
            
    csv_output.clear_output()
    
def execute_view_tag(widget, event, data):
    global centerlining_table
    global current
    global previous
    global selected_grade
    global tree
    global base_url
    global centerlining_workbook
    
    tag_name = tag_selection_input.v_model
    url = build_url(current, previous, selected_grade, tag_name, tree, base_url, centerlining_workbook)
    
    output.clear_output()
    with output:
        display(HTML(f'<a href="{url}" target="_blank">Click here to investigate {tag_name} in Workbench</a>'))

def update_top_table():
    global current_data
    global current_grade_metrics_datatable
    current_grade_metrics_datatable.headers = [{"text": col, "value": col} for col in current_data.columns]
    current_grade_metrics_datatable.items = current_data.to_dict('records')
    return

def update_table():
    global centerlining_table
    global current
    global previous
    global selected_grade
    global update_frequency
    global last_updated
    global g
    global dialog_grade_selection
    global tag_search_df
    global tree
    global current_grade_metrics_datatable
    global running_metrics
    global centerline_metadata
    
    while True:
        # If the table has been updated already and the other dialogs are closed, update both tables
        if len(centerlining_table) > 0 and dialog_grade_selection.v_model == False and dialog_tree_upload_or_manage.v_model == False and selected_grade != '':
            try:
                results = find_current_values(tag_search_df)
                current = results[0]
                last_updated = convert_date_format(results[1])
                centerlining_table = add_priority(current, previous, selected_grade)
                main_container.children[1].children = [f"Viewing Tree: {tree}. Viewing Grade: {selected_grade}. Last updated: {last_updated}"]
                g.update_grid_data(centerlining_table)
                current_data = find_current_metrics(running_metrics, centerline_metadata)
                current_data = current_data.applymap(replace_non_json_compliant_floats)
                update_top_table()
                

            except Exception as e:
                app_log.error(f'Error occurred in update_table 1: {e}')
                
        # If the table has NOT been updated already and the other dialogs are closed, update the current data table    
        elif len(centerlining_table) == 0 and dialog_grade_selection.v_model == False and dialog_tree_upload_or_manage.v_model == False and selected_grade != '':
            try:
                current_data = find_current_metrics(running_metrics, centerline_metadata)
                current_data = current_data.applymap(replace_non_json_compliant_floats)
                update_top_table()
                
            except Exception as e:
                app_log.error(f'Error occurred in update_table 2: {e}')
        # Otherwise, don't update anything... I.e. if select grade dialog is open
        else:
            pass
        
        # Wait for a specified period of time
        time.sleep(update_frequency)  

def execute_grade_selection(widget, event, data):
    global centerlining_table
    global current
    global previous
    global selected_grade
    global table_row
    global last_updated
    global g
    global first_text
    global grades
    global tag_search_df
    global tree
    global current_data
    global current_grade_metrics_datatable
    global centerline_metadata
    global running_metrics
    
    
    app_log.info('execute_grade_selection function started')
    please_wait.style_ = 'display: block'  # Make the please_wait visible when executing

    try:
        selected_grade = grade_selection_input.v_model
        results = find_current_values(tag_search_df)
        #app_log.info(f"Here are my Results: {results}")
        
        current = results[0]
        app_log.info(f"Current Results: {current}")
        
        last_updated = convert_date_format(results[1])
        previous = find_previous_run_data(selected_grade, centerline_metadata)
        app_log.info(f"Previous Results: {previous}")
        
        centerlining_table = add_priority(current, previous, selected_grade)

        g.update_grid_data(centerlining_table)

        
        main_container.children[1].children = [f"Viewing Tree: {tree}. Viewing Grade: {selected_grade}. Last updated: {last_updated}"]
        if isinstance(table_row.children[0], v.Html):
            table_row = v.Flex(xs12=True, class_='flex-grow-1', style_='min-height: 500px;', children=[g])
            main_container.children[2].children = [table_row]
        current_data = find_current_metrics(running_metrics, centerline_metadata)
        current_data = current_data.applymap(replace_non_json_compliant_floats)
        update_top_table()

    except Exception as e:
        app_log.error(f'Error occurred in execute_grade_selection: {e}')
        show_error_message(f"An error occurred: {e}")
    
    please_wait.style_ = 'display: none'
    dialog_grade_selection.v_model = False
    
def find_current_metrics(running_metrics, centerline_metadata):
    grade = grade_selection_input.v_model
    grade_condition = centerline_metadata['Name'] == 'Grade Keep Condition with Uptime Property'
    specific_grade = centerline_metadata['Path'].str.contains(f'.*{grade}', regex = True) 
    grade_condition_df = centerline_metadata[(grade_condition) & (specific_grade)].iloc[0]

    days = 31
    now_dt = datetime.now(timezone('US/Central'))
    start_date_dt = now_dt - timedelta(days=days)
    now_str = datetime.isoformat(now_dt, timespec='minutes')
    start_date_str = datetime.isoformat(start_date_dt, timespec='minutes')
    end_date_str = now_str

    calculation = f"$condition.keep('Grade Code', isEqualTo('{grade}')).removeShorterThan(30min).removeLongerThan(100d)"
    # Get previous run information
    data = spy.pull(items = grade_condition_df, 
                    calculation = calculation, 
                    start = start_date_str, 
                    end = end_date_str, 
                    quiet = True, 
                    shape = 'capsules',
                    #capsule_properties = ['Duration', 'Grade Code']
                   )
    
    # Convert 'Capsule Start' and 'Capsule End' to datetime objects in US/Central timezone
    data['Capsule Start'] = pd.to_datetime(data['Capsule Start']).dt.tz_convert('US/Central')
    data['Capsule End'] = pd.to_datetime(data['Capsule End']).dt.tz_convert('US/Central')

    # Calculate Duration
    data['Duration'] = data['Capsule End'] - data['Capsule Start']
    data['Duration'] = data['Duration'].apply(lambda x: f"{x.components.days} days {x.components.hours:02d}:{x.components.minutes:02d}")

    # Combine 'Capsule Start' and 'Capsule End' into a single column 'Start / End'
    data['Start / End; Results for Past Month'] = data['Capsule Start'].dt.strftime('%-m/%-d/%y %-I:%M %p') + ' - ' + data['Capsule End'].dt.strftime('%-m/%-d/%y %-I:%M %p')
    
    columns = ['Start / End; Results for Past Month', 'Duration', '% Uptime']

    
    data = data[columns]


    return data

## On Event Functions

In [ ]:
def find_all_grades_from_tag(widget, event, data):
    global unique_grades_list
    if grade_tag_item_selector.selected_value:
        pass
    else:
        show_error_message(f'You must search for and select a single tag.')
        return
    
    try:
        tag_df = pd.DataFrame([grade_tag_item_selector.selected_value])
        # Get the current timestamp (current time)
        current_time = datetime.now() - timedelta(minutes = 30)

        # Calculate the timestamp for one year ago
        one_year_ago = current_time - timedelta(days=365)
        one_year_pull = spy.pull(tag_df, start = one_year_ago, end = current_time, quiet = True)
        unique_grade_values = one_year_pull.iloc[:, 0].unique()
        unique_grades_list = [grade for grade in unique_grade_values if pd.notna(grade)]
        unique_grades_str = ", ".join([str(grade) for grade in unique_grade_values if pd.notna(grade)])
        final_sentence = f"You may proceed. Centerlining limits will be created for the following grades: {unique_grades_str}."
        with output_grades:
            display(final_sentence)
    except Exception as e:
        show_error_message(f'Error finding all unique Grades: {e}')
        
def find_all_values_from_tag(widget, event, data):
    if downtime_tag_item_selector.selected_value:
        pass
    else:
        show_error_message(f'You must search for and select a single tag.')
        return
    
    try:
        tag_df = pd.DataFrame([downtime_tag_item_selector.selected_value])
        # Get the current timestamp (current time)
        current_time = datetime.now() - timedelta(minutes = 30)

        # Calculate the timestamp for one year ago
        one_year_ago = current_time - timedelta(days=365)
        one_year_pull = spy.pull(tag_df, start = one_year_ago, end = current_time, quiet = True)
        unique_values = one_year_pull.iloc[:, 0].unique()
        unique_str = ", ".join([str(value) for value in unique_values if pd.notna(value)])
        final_sentence = f"You must specify the exact value below for what represents UPTIME. After entering the value, you may proceed. Found values: {unique_str}"
        with output_downtimes:
            display(final_sentence)
        input_value_uptime.disabled = False
    except Exception as e:
        show_error_message(f'Error finding all unique Grades: {e}')
        
def find_speed_tag(widget, event, data):
    global speed_tag_df
    if speed_tag_item_selector.selected_value:
        pass
    else:
        show_error_message(f'You must search for and select a single tag.')
        return
    
    try:
        tag_df = pd.DataFrame([speed_tag_item_selector.selected_value])
        
        speed_tag_df = spy.search(tag_df, quiet = True)
        final_sentence = f"Speed tag has been found. You may proceed."
        with output_speed_tag:
            display(final_sentence)
    except Exception as e:
        show_error_message(f'Error finding speed tag: {e}')

def update_grades_and_tags(widget, event, data):
    global grades
    global tags
    global tag_search_df
    global centerline_metadata
    global tree
    
    tree = tree_selection_input.v_model
    tag_search_path_raw_file = os.path.join(CENTERLINE_FOLDER_PATH, tree+'_centerline_tag_search.csv')
    metadata_path_raw_file = os.path.join(CENTERLINE_FOLDER_PATH, tree+'_centerline_metadata.csv')
    raw_upload_path = os.path.join(CENTERLINE_FOLDER_PATH, tree+'_raw_upload.csv')
    
    tag_search_df = pd.read_csv(tag_search_path_raw_file, usecols=lambda column: column not in ['Unnamed: 0'])
    centerline_metadata = pd.read_csv(metadata_path_raw_file, usecols=lambda column: column not in ['Unnamed: 0'])
    centerline_data = pd.read_csv(raw_upload_path, usecols=lambda column: column not in ['Unnamed: 0'])
    grades = list(centerline_metadata['Grade'].dropna().unique())
    
    # Filter rows where 'Path' column has 3 or more values split by '>>'
    filtered_df = centerline_metadata[centerline_metadata['Path'].str.count('>>') >= 2]

    # Get the unique values from the 'Asset' column
    unique_assets = filtered_df['Asset'].unique().tolist()
    
    tags = unique_assets # list(centerline_data['Tag Name'])

    grade_selection_input.items = grades
    tag_selection_input.items = tags

def select_grade_click(widget, event, data):
    dialog_grade_selection.v_model = True


def view_tag_click(widget, event, data):
    dialog_view_tag.v_model = True
    
def create_or_manage_click(widget, event, data):
    dialog_tree_upload_or_manage.v_model = True
    
def help_csv_format_click(widget, event, data):
    help_dialog.v_model = True
    
def change_view_upload_or_manage(widget, event, data):
    selection = upload_or_manage_selector.v_model
    if selection == 'Manage':
        dialog_tree_upload_or_manage.children = dialog_tree_manage.children
    else:
        dialog_tree_upload_or_manage.children = dialog_tree_upload.children
        
def delete_tree_select(widget, event, data):
    execute_delete_tree_button.disabled = False

def delete_tree_click(widget, event, data):
    delete_tree_output.clear_output()
    
    tree_to_delete = delete_tree_selection_input.v_model
    try:
        with delete_tree_output:
            print(f"Deleting tree: {tree_to_delete}")
            delete_tree(tree_to_delete)
        execute_delete_tree_button.disabled = True
        centerlining_tree_selections = get_centerlining_sets(CENTERLINE_FOLDER_PATH)
        tree_selection_input.items = centerlining_tree_selections
        delete_tree_selection_input.items = centerlining_tree_selections
        
    except Exception as e:
        show_error_message(f"Error deleting Tree: {e}")
    
# Create a function to handle file upload
def on_upload_change(change):
    global csv_file_upload
    try:
        
        # Get the name of the first (and only) uploaded file
        file_name = list(change['new'].keys())[0]
        
        # Get the content of the uploaded file
        content = change['new'][file_name]['content']

        # Convert the bytes to a string
        content_str = content.decode()

        # Convert the string to a pandas DataFrame
        upload = pd.read_csv(StringIO(content_str))
        
        # Validate columns
        is_valid, validation_message = validate_columns(upload)
        if not is_valid:
            show_error_message(validation_message)
            file_upload.value = {}
            
            return
            
        csv_file_upload = pd.read_csv(StringIO(content_str))

        # Display the DataFrame
        csv_output.clear_output()
        with csv_output:
            display(csv_file_upload)
        execute_new_tree_button.disabled = False
            
    except Exception as e:
        file_upload.value = {}
        csv_output.clear_output()
        with csv_output:
            display(f"An error occurred: {e}")
        
def execute_new_tree_click(widget, event, data):
    name_of_tree = asset_tree_name_input.v_model
    if name_of_tree == '':
        show_error_message('Name for the Centerlining Dataset needs to be set.')
    elif not check_all_inputs_specified(input_widgets):
        show_error_message('All configuration items must be set.')
    else:

        create_centerlining_tree()

# Event Handlers

In [ ]:
execute_view_tag_button.on_event('click', execute_view_tag)
view_tag_in_wb_button.on_event('click', view_tag_click)
create_or_manage_tree_button.on_event('click', create_or_manage_click)
select_grade_button.on_event('click', select_grade_click)
execute_grade_selection_button.on_event('click', execute_grade_selection)
tree_selection_input.on_event('change', update_grades_and_tags)
file_upload.observe(on_upload_change, names='value')
execute_new_tree_button.on_event('click', execute_new_tree_click)
execute_delete_tree_button.on_event('click', delete_tree_click)
upload_or_manage_selector.on_event('change', change_view_upload_or_manage)
help_csv_format_button.on_event('click', help_csv_format_click)
delete_tree_selection_input.on_event('change', delete_tree_select)
find_all_modes_button.on_event('click', find_all_values_from_tag)
find_all_grades_button.on_event('click', find_all_grades_from_tag)
find_speed_tag_button.on_event('click', find_speed_tag)



# Display App & Create Thread

In [ ]:
# To update the table
if update:
    # Create a thread to run the update_table function in the background
    update_thread = threading.Thread(target=update_table)

    # Start the thread
    update_thread.start()

display(app)